In [2]:
from __future__ import print_function 
import numpy as np
import pandas as pd
import tensorflow as tf 

## Load the dataset (without word embedding and one-hot labels)

In [3]:
label_num = 8

labels = ['ooo', 'Too', 'oEo', 'ooD', 'TEo', 'ToD', 'oED', 'TED']
label_dict = {labels[i]: i for i in range(8)}
index_dict = {i: labels[i] for i in range(8)}
print(label_dict)
print(index_dict)

{'ooo': 0, 'Too': 1, 'oEo': 2, 'ooD': 3, 'TEo': 4, 'ToD': 5, 'oED': 6, 'TED': 7}
{0: 'ooo', 1: 'Too', 2: 'oEo', 3: 'ooD', 4: 'TEo', 5: 'ToD', 6: 'oED', 7: 'TED'}


In [4]:
f = np.load('corpus_all_9999.npz')

In [5]:
# retrieve raw datasets (without embeddings) from the data file

train_doc = f['train_doc']
valid_doc = f['valid_doc']
test_doc = f['test_doc']

## Check classes distributions 

In [6]:
def get_class_distribution(data, index_dict):
    d = {index_dict[key]:0 for key in index_dict}
    for doc in data:
        d[index_dict[doc[1]]] += 1
    return d

In [7]:
train_classes_distri = get_class_distribution(train_doc, index_dict)
print(train_classes_distri)

{'ooo': 847, 'Too': 295, 'oEo': 132, 'ooD': 116, 'TEo': 24, 'ToD': 119, 'oED': 20, 'TED': 27}


In [8]:
validation_classes_distri = get_class_distribution(valid_doc, index_dict)
print(validation_classes_distri)

{'ooo': 139, 'Too': 46, 'oEo': 17, 'ooD': 24, 'TEo': 4, 'ToD': 12, 'oED': 2, 'TED': 4}


In [9]:
test_classes_distri = get_class_distribution(test_doc, index_dict)
print(test_classes_distri)

{'ooo': 139, 'Too': 40, 'oEo': 21, 'ooD': 24, 'TEo': 7, 'ToD': 14, 'oED': 2, 'TED': 3}


In [10]:
count = 0
for key in train_classes_distri:
    count += train_classes_distri[key]
    
for key in train_classes_distri:
    print(train_classes_distri[key]/count)

0.5360759493670886
0.18670886075949367
0.08354430379746836
0.07341772151898734
0.015189873417721518
0.07531645569620253
0.012658227848101266
0.01708860759493671


## Build the vocabulary

In [11]:
from collections import Counter 

In [12]:
# API for corpuse format conversion 

def map_word_and_index(input_doc, top=10000):
    """ construct words and indices mapping dictionaries 
    """
    counts_new = []            
    words_count_list = Counter([word for doc in input_doc for sent in doc[0] for word in sent]).most_common(top)
    word2index = {item[0]: index for index, item in enumerate(words_count_list, 1)}
    index2word = {index: item[0] for index, item in enumerate(words_count_list, 1)}
    return word2index, index2word

def convert2words(doc):
    """ flatten a document into a list of words 
    """
    return [word for sent in doc for word in sent]

def doc2index(doc, word2index):
    """ map words to indices for the flatten document 
    """
    return [[word2index[word]] for word in doc]

def convert_corpus(corpus, word2index):
    corpus_words = [(convert2words(doc[0]), doc[1]) for doc in corpus]
    return np.asarray([doc2index(doc[0], word2index) for doc in corpus_words])

In [14]:
word2index, index2word = map_word_and_index(train_doc, 10000)

In [15]:
# datasets with each document as a list of word indices (no sentence structure)

train_wordasindex = convert_corpus(train_doc, word2index)
valid_wordasindex = convert_corpus(valid_doc, word2index)
test_wordasindex = convert_corpus(test_doc, word2index)

## Embed words 

In [16]:
from gensim.models import Word2Vec

Using TensorFlow backend.


In [17]:
model = Word2Vec.load("word2vec_model")

In [18]:
def embed_from_index(model, idx):
    """ apply embedding to a word represented as an index 
    """
    if idx != 0:
        return model.wv[index2word[idx]]
    else:
        return np.zeros(shape=model.wv[index2word[1]].shape)

def embed_corpus(corpus, word2index, model):
    """ apply word embedding to the dataset 
    """
    temp = list(corpus)
    return np.asarray([[embed_from_index(model, idx) for sent in doc for idx in sent] for doc in temp])

In [19]:
# embed words with trained Word2Vec model (with sentence structure)
# This part is only for hierarchical model which is subject to future exploration

# train_wordasindex_embed = embed_corpus(train_doc, word2index, model)

# valid_wordasindex_embed = embed_corpus(valid_doc, word2index, model)

# test_wordasindex_embed = embed_corpus(test_doc, word2index, model)

## Convert labels 

In [20]:
def encode_label(label, size):
    """ one-hot encode the given label 
    """
    l = [0]*size
    l[label] = 1
    return l

def encode_class(corpus, size):
    """ apply one-hot encoding to the dataset labels 
    """
    return np.asarray([encode_label(doc[1], size) for doc in corpus])

In [21]:
train_label = encode_class(train_doc, 8)
valid_label = encode_class(valid_doc, 8)
test_label = encode_class(test_doc, 8)

In [22]:
# maximum doc length in terms of number of words

min_count = 100
max_count = 0

for doc in valid_wordasindex:
    if len(doc) < min_count:
        min_count = len(doc)
    if len(doc) > max_count:
        max_count = len(doc)

print(min_count)
print(max_count)


100
6017


## Save the flattened dataset and reload

In [29]:
np.savez('corpus_wordasindex_all_9999', train_wordasindex=train_wordasindex,
         valid_wordasindex=valid_wordasindex, test_wordasindex=test_wordasindex)

In [31]:
# reload the flattened dataset, use it after you save it once 

d = np.load('corpus_wordasindex_all_9999.npz')

train_wordasindex = d['train_wordasindex']
valid_wordasindex = d['valid_wordasindex']
test_wordasindex = d['test_wordasindex']

## Classification model 

In [32]:
max_features = 10000

learning_rate = 0.001
maxlen = 6700
batch_size = 50
total_batch = int(train_wordasindex.shape[0]/batch_size)
input_dims = 100
num_hidden= 50
epochs = 100

index=0

### Padding with 0s

In [33]:
def pad_with_zeros(sequence, maxlen):
    """ pad text body with preceding 0s 
    """
    if len(sequence) > maxlen:
        raise Exception
    else:
        return [[0]]*(maxlen-len(sequence))+sequence

def corpus_pad_with_zeros(corpus, maxlen):
    """ apply zero padding for the whole dataset 
    """
    return np.asarray([pad_with_zeros(sent, maxlen) for sent in corpus])

In [34]:
# zero padding on the texts

train_corpus = corpus_pad_with_zeros(train_wordasindex, maxlen)

valid_corpus = corpus_pad_with_zeros(valid_wordasindex, maxlen)

test_corpus = corpus_pad_with_zeros(test_wordasindex, maxlen)

In [35]:
# embed the dataset (without sentence structure)

train_corpus_embed = embed_corpus(train_corpus, word2index, model)

valid_corpus_embed = embed_corpus(valid_corpus, word2index, model)

test_corpus_embed = embed_corpus(test_corpus, word2index, model)

In [36]:
def next_batch(data, index, size):
    """ return next batch in format: index, x batch, y batch
    """
    if index + size <= data[0].shape[0]:
        return index+size, data[0][index:index+size], data[1][index:index+size]
    else:
        return index+size-data[0].shape[0], np.concatenate((data[0][index:],data[0][:index+size-data[0].shape[0]]), 0), \
    np.concatenate((data[1][index:],data[1][:index+size-data[1].shape[0]]), 0)

def length(sequence):
    used = tf.sign(tf.reduce_max(tf.abs(sequence), reduction_indices=2))
    length = tf.reduce_sum(used, reduction_indices=1)
    length = tf.cast(length, tf.int32)
    return length

## Model (LSTM)

For recurrent neural networks, tensorflow wants a data format of [Batch Size, Sequence Length, Input Dimension].

In [37]:
x = tf.placeholder(tf.float32, [None, maxlen, input_dims])
y = tf.placeholder(tf.float32, [None, 8])

Constructing a LSTM cell, simply call the "tf.contrib.rnn.LSTMCell" function with given arguments for size of the hidden state, "state_is_tuple=True" will get both the hidden state and the cell state.

You can also easily construct RNN variants, for example, call "tf.conctrib.rnn.GRUCell" for a GRU network, all the rest is the same. 

In [38]:
with tf.variable_scope('cell_def', reuse=True):
    cell_fw = tf.contrib.rnn.LSTMCell(num_hidden, state_is_tuple=True)
    # cell_bw = tf.contrib.rnn.LSTMCell(num_hidden, state_is_tuple=True)

Building a forward LSTM network, call "tf.nn.dynamic_rnn" with the constructed LSTM cell and input tensor, specifying "sequence_length" will dynamically unroll the network to a matching length to the current input during computation. "val" will be a sequence of outputs and "state" will be the last hidden state from the LSTM.

In [39]:
with tf.variable_scope('lstm_def'):
    val, state = tf.nn.dynamic_rnn(cell_fw, x, dtype=tf.float32, sequence_length=length(x))    # val are the hidden states

apply nonlinear transformation to the LSTM output (we choose mean of the hidden states here, you can try out different variants such as the final hidden state, with or without transformation, etc) and render for classification using softmax.

In [40]:
W = tf.Variable(tf.truncated_normal(shape=[50, 32]))
b = tf.Variable(tf.constant(0.0, shape=[32]))

V = tf.Variable(tf.truncated_normal(shape=[32, 8]))
c = tf.Variable(tf.constant(0.0, shape=[8]))

# dropout_rate = tf.placeholder(tf.float32)

h = tf.nn.relu(tf.matmul(tf.reduce_mean(val, axis=1), W) + b)
# h = tf.nn.relu(tf.matmul(state[1], W) + b)
u = tf.matmul(h, V) + c
p = tf.nn.softmax(u)
pred = tf.argmax(p, 1)
loss = tf.reduce_mean(tf.reduce_sum(-tf.cast(y, tf.float32)*tf.log(p), 1))

In [41]:
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss)

In [42]:
accuracy = tf.reduce_mean(tf.cast(tf.equal(pred, tf.argmax(y, 1)), tf.float32))

In [43]:
# instantiate model saver to save the model in a session 
saver = tf.train.Saver()

### Training 

In [44]:
sess = tf.InteractiveSession()

In [45]:
init = tf.global_variables_initializer()
sess.run(init)

In [ ]:
for i in range(epochs+1):
    xloss = 0
    
    for j in range(total_batch):
        # need to incoporate y in the batches and expand to 8 classes 
        index, x_, y_ = next_batch((train_corpus_embed, train_label), index, batch_size)
        _, xloss, acc_train = sess.run([optimizer, loss, accuracy], feed_dict={x: x_, y: y_})
        
        if j % 10 == 0:
            print("epoch %d, run %d, loss %g" % (i, j, xloss))
            
    if i % 2 == 0:
        acc_val = sess.run(accuracy, feed_dict={x:valid_corpus_embed, y:valid_label})
        print("epoch %d, Training acc: %g, Validation acc: %g " % (i, acc_train, acc_val))
        
save_path = saver.save(sess, "models/model.ckpt")
print("Model saved in file: %s" % save_path)

sess.close()

epoch 0, run 0, loss 2.08369
epoch 0, run 10, loss 1.92603
epoch 0, run 20, loss 1.80858
epoch 0, run 30, loss 1.62792
epoch 0, Training acc: 0.58, Validation acc: 0.556452 
epoch 1, run 0, loss 1.78788
epoch 1, run 10, loss 1.84149
epoch 1, run 20, loss 1.70693
epoch 1, run 30, loss 1.44119
epoch 2, run 0, loss 1.33701
epoch 2, run 10, loss 1.58
epoch 2, run 20, loss 1.37917
epoch 2, run 30, loss 1.14554
epoch 2, Training acc: 0.7, Validation acc: 0.556452 
epoch 3, run 0, loss 1.51547
epoch 3, run 10, loss 1.52064
epoch 3, run 20, loss 1.43776
epoch 3, run 30, loss 1.27195
epoch 4, run 0, loss 1.32246
epoch 4, run 10, loss 1.37758
epoch 4, run 20, loss 1.37754
epoch 4, run 30, loss 1.36137
epoch 4, Training acc: 0.54, Validation acc: 0.564516 
epoch 5, run 0, loss 1.14413
epoch 5, run 10, loss 1.25797
epoch 5, run 20, loss 1.34334
epoch 5, run 30, loss 1.5049
epoch 6, run 0, loss 1.25913
epoch 6, run 10, loss 1.45705
epoch 6, run 20, loss 1.30053
epoch 6, run 30, loss 1.55018
epoch 6

### Test

In [ ]:
with tf.Session() as sess:
    # Restore variables from disk.
    saver.restore(sess, "models/model.ckpt")
    print("Model restored.")

    acc = sess.run(accuracy, feed_dict={x:test_corpus_embed, y:test_label})
    print("Test acc: %g" % (acc))

## Questions

- What are the benefits and downsides of the RNN-based representation over the bag of words representation used last week? 
- How would availability of data affect your answer?
- One possible architectural variant is to use only the final hidden state of the RNN as the document representation (i.e., x) rather than the average of the hidden states over time. How does this work? What are the potential benefits and downsides to this representation?
- Try different RNN architectures, e.g., simple Elman RNNs or GRUs or LSTMs. Which ones work best?
- What happens if you use a bidirectional LSTM (i.e., the dashed arrows in the figure)?

## Playground

### Bidirectional LSTM 

In [ ]:
x = tf.placeholder(tf.float32, [None, maxlen, input_dims])
y = tf.placeholder(tf.float32, [None, 8])

Bidirectional LSTM needs a forward LSTM cell and a backward LSTM cell (they are the same here). 

In [ ]:
with tf.variable_scope('cells_def', reuse=True):
    cell_fw = tf.contrib.rnn.LSTMCell(num_hidden, state_is_tuple=True)
    cell_bw = tf.contrib.rnn.LSTMCell(num_hidden, state_is_tuple=True)

Build a bidirectional LSTM network is as easy as calling the "tf.bidirectional_dynamic_rnn" function with the foward and backward cells, the returned outputs and states are both a tuple of two, each contains the sequence of output states and the final hidden state for the corresponding direction.

In [ ]:
with tf.variable_scope('bidirlstm_def'):
    outputs, states = tf.nn.bidirectional_dynamic_rnn(cell_fw, cell_bw, x, dtype=tf.float32, sequence_length=length(x))    # val are the h_ts 

You can choose how to use the outputs and hidden states from both directions. Here we are only using the output states and we take the mean between the forward and backward directions. 

In [ ]:
val = (outputs[0] + outputs[1])/2

Up to this point, you should get the new representation of the document, classify it through further nonlinear transformation and softmax.

In [ ]:
W = tf.Variable(tf.truncated_normal(shape=[50, 32]))
b = tf.Variable(tf.constant(0.0, shape=[32]))

V = tf.Variable(tf.truncated_normal(shape=[32, 8]))
c = tf.Variable(tf.constant(0.0, shape=[8]))

# dropout_rate = tf.placeholder(tf.float32)

h = tf.nn.relu(tf.matmul(tf.reduce_mean(val, axis=1), W) + b)
# h = tf.nn.relu(tf.matmul(state[1], W) + b)
u = tf.matmul(h, V) + c
p = tf.nn.softmax(u)
pred = tf.argmax(p, 1)
loss = tf.reduce_mean(tf.reduce_sum(-tf.cast(y, tf.float32)*tf.log(p), 1))

optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss)
accuracy = tf.reduce_mean(tf.cast(tf.equal(pred, tf.argmax(y, 1)), tf.float32))
saver = tf.train.Saver()

### Training 

In [ ]:
sess = tf.InteractiveSession()

In [ ]:
init = tf.global_variables_initializer()
sess.run(init)

In [ ]:
for i in range(epochs+1):
    xloss = 0
    
    for j in range(total_batch):
        # need to incoporate y in the batches and expand to 8 classes 
        index, x_, y_ = next_batch((train_corpus_embed, train_label), index, batch_size)
        _, xloss, acc_train = sess.run([optimizer, loss, accuracy], feed_dict={x: x_, y: y_})
        
        if j % 10 == 0:
            print("epoch %d, run %d, loss %g" % (i, j, xloss))
            
    if i % 2 == 0:
        acc_val = sess.run(accuracy, feed_dict={x:valid_corpus_embed, y:valid_label})
        print("epoch %d, Training acc: %g, Validation acc: %g " % (i, acc_train, acc_val))
        
save_path = saver.save(sess, "models/bidir_model.ckpt")
print("Model saved in file: %s" % save_path)

sess.close()

### Tests

In [ ]:
with tf.Session() as sess:
    # Restore variables from disk.
    saver.restore(sess, "models/bidir_model.ckpt")
    print("Model restored.")

    acc = sess.run(accuracy, feed_dict={x:test_corpus_embed, y:test_label})
    print("Test acc: %g" % (acc))